In [13]:
!pip install --upgrade "qiskit[all]" qiskit-nature qiskit-aer
!pip install "qiskit<2" 
!pip install qiskit-optimization
!pip install qiskit-finance


  Using cached qiskit-2.1.1-cp39-abi3-win_amd64.whl.metadata (13 kB)
Using cached qiskit-2.1.1-cp39-abi3-win_amd64.whl (7.2 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 1.4.3
    Uninstalling qiskit-1.4.3:
      Successfully uninstalled qiskit-1.4.3



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached qiskit-1.4.3-cp39-abi3-win_amd64.whl.metadata (12 kB)
Using cached qiskit-1.4.3-cp39-abi3-win_amd64.whl (6.5 MB)
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.1.1
    Uninstalling qiskit-2.1.1:
      Successfully uninstalled qiskit-2.1.1



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ------------------------------------- 949.2/949.2 kB 23.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 54.2 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15703 sha256=325c2a3f54fb871458f7e51b2c6cbc8c43ad074a59177a828f9fd6d6d378309a
  St

  DEPRECATION: Building 'multitasking' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'multitasking'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'fastdtw' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'fastdtw'. Discussion can be found at https://github.com/pypa/pip/issues/6334

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To up

In [15]:
import numpy as np

from qiskit_finance.applications import PortfolioOptimization
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver, SamplingVQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit_aer.primitives import Sampler

# ----- Your data -----
mu = np.array([0.10, 0.20, 0.15, 0.12])
sigma = np.array([
    [ 0.005, -0.010,  0.004, -0.002],
    [-0.010,  0.040, -0.002,  0.004],
    [ 0.004, -0.002,  0.023,  0.002],
    [-0.002,  0.004,  0.002,  0.015]
])
q = 0.5        # risk factor (λ)
budget = 2     # select exactly 2 assets

# ----- Build QuadraticProgram from the Finance app -----
portfolio = PortfolioOptimization(
    expected_returns=mu,
    covariances=sigma,
    risk_factor=q,
    budget=budget
)
qp = portfolio.to_quadratic_program()   # penalty handled internally

def show(result, title):
    x = result.x.astype(int)
    print(f"\n== {title} ==")
    print("Selection (1=invest):", x, "  cost:", result.fval)
    print("Picked asset indices:", [i for i, v in enumerate(x) if v == 1])
    # optional: report portfolio stats
    print("Expected return:", float(mu @ x))
    print("Variance:", float(x @ sigma @ x))

# ----- Classical reference (exact on CPU) -----
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())
res_exact = exact.solve(qp)
show(res_exact, "Classical Exact (NumPyMinimumEigensolver)")

# ----- QAOA (Sampler primitive) -----
sampler = Sampler()
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=250), reps=3)
qaoa_solver = MinimumEigenOptimizer(qaoa)
res_qaoa = qaoa_solver.solve(qp)
show(res_qaoa, "QAOA")

# (Optional) SamplingVQE baseline
svqe = SamplingVQE(sampler=sampler, optimizer=COBYLA(maxiter=400))
svqe_solver = MinimumEigenOptimizer(svqe)
res_svqe = svqe_solver.solve(qp)
show(res_svqe, "SamplingVQE")



== Classical Exact (NumPyMinimumEigensolver) ==
Selection (1=invest): [0 1 1 0]   cost: -0.3205
Picked asset indices: [1, 2]
Expected return: 0.35
Variance: 0.059

== QAOA ==
Selection (1=invest): [0 1 1 0]   cost: -0.3205
Picked asset indices: [1, 2]
Expected return: 0.35
Variance: 0.059


TypeError: SamplingVQE.__init__() missing 1 required positional argument: 'ansatz'

In [ ]:
-